# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik


The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Write all your answers and the code in the appropriate boxes below.

---

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for phrases affect both the model and the learned information about similarities.  

Note that this lab uses a code from `dist_erk.py`, which contains functions that highly resemble those shown during the lecture. In the end, you can use either of the functions (from the lecture / from the file) to solve the tasks.

In [1]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

**Important**: All necessary files which are used in this notebook are available on mlt-gpu, check `/srv/data/computational-semantics-assignment-02`.

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus (`wikipedia.txt`. This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/).

When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below.  
<!-- <It may already exist in `/opt/mlt/courses/cl2015/a5`.> -->


In [2]:
corpus_dir = '/srv/data/computational-semantics-assignment-02/'

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py` earlier, and they largely resemble functions shown during the lecture.

In [3]:
numdims = 1000
svddim = 5

# which words to use as targets and context words?
# we need to count the words and keep only the N most frequent ones
# which function would you use here with which variable?
ktw = do_word_count(corpus_dir, numdims)

wi = make_word_index(ktw)  # word index
words_in_order = sorted(wi.items(), key=lambda x: x[1])  # sorted words

reading file wikipedia.txt


In [4]:
# create different spaces (the original matrix space, the ppmi space, the svd space)
# which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir, wi, numdims)
print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k, wi)
print('svd transform')
svdspace_1k = svd_transform(ppmispace_1k, numdims, svddim)
print('done.')

create count matrices
reading file wikipedia.txt
ppmi transform
svd transform
done.


In [5]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])

house: [2554 3774 3105  567  962  631  443  185  311  189  131   28   93  169
   81  125  151  408  194   90   79   29  217  184   62   15   31   70
   10    1   41   21    1   31   37    1   30    5   25    7    3   20
   11    1   32   36    2    5   66    4    0   46    8   18   28    0
   20    7    8   16   10   40    0  175   10    2    7   19    1  174
   11    3    1    6    0    0    0   10    9   11    7   24    4    4
   14   23   58    7    0   10    2    3   10    6   18    6   13    3
   22    0    3    5    3    7   14    3   40   20   19   15    6    8
   24    4    5    1   19    0    3    1    0   14    0   14   53    7
    7   11    6    5    5    4   12    6   53    1    1  433    4    0
    5    7    7   12    1    1    3    4   17    8   16    1    2   31
    1   12   14    1   44    6   14    9   38    7    2    6    8    1
   10    6   10    1    9    7    9    4    3   10    0   11    3    2
    0    2   11   37    2    0    2    1    5    9   10   16   88    6

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. All matrices are available on mlt-gpu: `ktw_wikipediaktw.npy`, `raw_wikipediaktw.npy`, `ppmi_wikipediaktw.npy`, `svd50_wikipedia10k.npy`. Make sure they are in your path, because they will be loaded below.

In [6]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k = np.load(f'{corpus_dir}ktw_wikipediaktw.npy', allow_pickle=True)
space_10k = np.load(f'{corpus_dir}raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load(f'{corpus_dir}ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k = np.load(f'{corpus_dir}svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [7]:
# testing semantic space
print('house:', space_10k['house'])

house: [2554 3774 3105 ...    0    0    0]


## 3. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture (the one from [2]). For more information, please read the papers.

The following code will transform similarity scores into a Python-friendly format:

In [8]:
word_pairs = []  # test suit word pairs
semantic_similarity = []
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))

print('number of available words to test:', len(test_vocab - (test_vocab - set(ktw))))
print('number of available word pairs to test:', len(word_pairs))
list(zip(word_pairs, visual_similarity, semantic_similarity))

number of available words to test: 12
number of available word pairs to test: 774


[(('stick', 'sword'), 3.4, 2.6),
 (('cabin', 'cabinet'), 1.75, 2.25),
 (('chicken', 'sparrow'), 3.0, 4.5),
 (('bag', 'gate'), 1.0, 1.5),
 (('bull', 'trumpet'), 1.0, 1.25),
 (('balloon', 'brick'), 1.0, 1.0),
 (('helicopter', 'jet'), 2.6, 4.6),
 (('cape', 'spider'), 1.0, 1.2),
 (('boots', 'mouse'), 1.0, 1.0),
 (('kite', 'marble'), 1.0, 1.0),
 (('box', 'horse'), 1.0, 1.0),
 (('doll', 'elephant'), 1.0, 1.0),
 (('brick', 'cannon'), 1.0, 1.0),
 (('ruler', 'turkey'), 1.0, 1.0),
 (('pan', 'pot'), 4.0, 4.75),
 (('bottle', 'orange'), 1.0, 1.25),
 (('plate', 'rock'), 1.75, 1.5),
 (('subway', 'wall'), 1.25, 1.25),
 (('cup', 'level'), 1.0, 1.2),
 (('book', 'submarine'), 1.0, 1.0),
 (('bus', 'gun'), 1.0, 1.0),
 (('inn', 'telephone'), 1.0, 1.4),
 (('bike', 'hook'), 1.0, 1.0),
 (('building', 'car'), 1.2, 1.6),
 (('airplane', 'truck'), 1.2, 3.4),
 (('cat', 'rabbit'), 2.75, 4.5),
 (('cedar', 'pine'), 4.5, 4.75),
 (('corn', 'oak'), 1.75, 2.25),
 (('chapel', 'magazine'), 1.0, 1.0),
 (('book', 'bureau'), 1

Now we are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [9]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [10]:
raw_similarities = [cosine(w1, w2, space_10k) for w1, w2 in word_pairs]
ppmi_similarities = [cosine(w1, w2, ppmispace_10k) for w1, w2 in word_pairs]
svd_similarities = [cosine(w1, w2, svdspace_10k) for w1, w2 in word_pairs]

Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [11]:
simdict = {"raw": raw_similarities, "ppmi": ppmi_similarities, "svd": svd_similarities}

In [12]:
for name, sims in simdict.items():
    rho, pval = stats.spearmanr(semantic_similarity, sims)
    print(f"""Original similarity and {name} similarity
Rho: {rho:4f}
P-value: {pval:4f}
""")

Original similarity and raw similarity
Rho: 0.152236
P-value: 0.000021

Original similarity and ppmi similarity
Rho: 0.454744
P-value: 0.000000

Original similarity and svd similarity
Rho: 0.423221
P-value: 0.000000



**Your answer should go here:**

With consulting Jurafsky and Martin's 'Speech and Language Processing', we could conclude that raw frequency is less effective than ppmi, because raw frequency is not discriminative enough, since it doens't discount words such as 'it', 'they', 'the', which are not very informative, whilst ppmi helps show those situations with informative co-occurrence.

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [13]:
for name, sims in simdict.items():
    rho, pval = stats.spearmanr(visual_similarity, sims)
    print(f"""Visual similarity and {name} similarity
Rho: {rho:4f}
P-value: {pval:4f}
""")

Visual similarity and raw similarity
Rho: 0.121179
P-value: 0.000729

Visual similarity and ppmi similarity
Rho: 0.383771
P-value: 0.000000

Visual similarity and svd similarity
Rho: 0.309655
P-value: 0.000000



**Your answer should go here:**

The scores go from best to worst (PPMI, SVD, raw) in the same way as above, although in this case they are overall lower. There has been a bit said about this at last week’s lab, about the way in which a pair of words can either have a low semantic similarity score and low visual similarity score, or the other way around. The other group mentioned how, in the case of low visual similarity and high semantic similarity, this might be due to the fact that people are more “lenient” when it comes to finding semantic similarity, but more scrutinizing in the case of visual similarity (lion and cat might score high for semantic similarity, since they are both felines, but might score lower visually, because when you are put in front of two “items” from the same category, you will notice their differences and what gives them an individual look). However, this discussion was centered heavily around human perception. I would argue that there would be a large amount of cases in which the scores would present themselves in the other way around, simply because an AI would see two “similar” images in terms of colour, shape, size, positioning, shading etc, but would not be able to distinguish the actual “representation” of what it sees, thus deeming the two highly similar. A good example for this would be an image of the sun, next to an image of a peach.

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions.

For example, we can perform `king - man` and add the resulting vector to `woman` and we hope to get the vector for `queen`. Also, what would be the result of `stockholm - sweden + denmark`? Why? **[3 marks]**

If you want to learn more about vector differences between words (and words in analogy relations), check [this paper](https://aclanthology.org/P16-1158.pdf).

**Your answer should go here:**

Copenhagen, because stockholm is a capital city, and if we subtract the country which determines the capital, and "add" another country, then the answer would be the corresponding capital city for that country

Here is some helpful code that allows us to calculate such comparisons.

In [14]:
from scipy.spatial import distance


def normalize(vec):
    return vec / veclen(vec)


def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1 - distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key=lambda p: p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [15]:
short = normalize(svdspace_10k['short'])
light = normalize(svdspace_10k['light'])
long = normalize(svdspace_10k['long'])
heavy = normalize(svdspace_10k['heavy'])

find_similar_to(light - (heavy - long), svdspace_10k)[:10]

[('long', 0.8733111261346901),
 ('above', 0.8259671977311955),
 ('around', 0.8030776291120685),
 ('sun', 0.7692439111243973),
 ('just', 0.7678481974778111),
 ('wide', 0.767257431992253),
 ('each', 0.7665960260861158),
 ('circle', 0.7647746702909336),
 ('length', 0.7601066921319761),
 ('almost', 0.7542351860536628)]

**Your answer should go here:**

It seems like in the light - (heavy - long) equation, the results at the top of the list should be words whose value are closest to heavy - long (which we thought would be wide), and then subtract light from wide, which would give various results, depending on how the sense of 'light' is interpreted (not dark or not heavy). 'Long' being at the top of the list was surprising, since it is subtracted originally. 'above' and 'around' seem unclear, but then we have 'sun', which makes sense, considering that the sun could be described as being heavy and wide, and then the above words make sense as well, in connection with sun.

Find 5 similar pairs of pairs of words and test them. Hint: Google for `word analogies examples`. You can also construct analogies that are less lexical but more grammatical, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) from [3]. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [16]:
black = normalize(svdspace_10k['black'])
white = normalize(svdspace_10k['white'])
pretty = normalize(svdspace_10k['pretty'])
beautiful = normalize(svdspace_10k['beautiful'])
leave = normalize(svdspace_10k['leave'])
left = normalize(svdspace_10k['left'])
here = normalize(svdspace_10k['here'])
there = normalize(svdspace_10k['food'])
love = normalize(svdspace_10k['love'])
like = normalize(svdspace_10k['like'])

find_similar_to(beautiful - (pretty + like), svdspace_10k)[:5]

[('census2', 0.10841408026301902),
 ('susquehanna', 0.06846910991214805),
 ('cambria', 0.06005636859121455),
 ('presbyterian', 0.05958665621532133),
 ('seminary', 0.05743497102845396)]

The results were pretty unexpected mostly, as we couldn't see the relevance between the given 'equation' and the words in the output most of the times. The above output we think is a pretty good example.

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to look at how different semantic composition models, introduced in [2] correlate with human judgements. The file with the dataset `mitchell_lapata_acl08.txt` is included with this notebook (we also used it in the class).

---

Explanation of the task from Discord channel:

**What are we trying to achieve?**  
We want to create models, which can automatically capture differences between meaning of different phrases. These models should also be as good as we are (humans) in this task. Check example (1) from Mitchell/Lapata paper, it has two sentences which share the same words, but their meaning is completely different. We as humans can clearly see this difference, but how could a machine capture it?

It is intuitive that in order to get a meaning of a phrase, we might combine meaning of individual words in this phrase somehow, but how? First, we represent each word with the frequency vector based on the semantic space that we built before (e.g, frequency space, ppmi space, svd space). In other words, each word's meaning is represented by the number of times other words occur in the context, defined by window size. Now, we have a vector for `discussion` and a vector for `thrive`.

_How do we combine these vectors to get a single vector for the phrase `discussion thrive`?_

Such methods of combining meaning vectors into a single item are called _semantic composition methods_ (literally, because we compose semantic meaning of the phrase from its individuals). During the lecture, we tried different semantic composition methods: additive, multiplicative, combined.

Let's say we multiplied the vectors (went with the multiplicative method) and now we have one vector for our phrase.

Remember, we want to have a model that captures differences between the phrases; it means that if `discussion thrive` is our reference phrase, we need to have a different phrase (high or low similarity phrase) to compare it against the reference one. How do we get this other phrase? Well, this other phrase can be either very similar to the reference or not similar at all, right? Let's say we decided to go with the second option and made/constructed a phrase `discussion digress`, which we know is very dissimilar to the reference phrase. We label this pair of phrases as having a low similarity, e.g., `low` in `hilo`. We can also create a different phrase (e.g., `discussion develop`) and use it as a high similarity phrase when paired with our reference phrase, right? This then would be labeled as `high` in `hilo`. This is what `hilo` in the dataset stands for: known information about how similar the reference phrase and the landmark phrase are.

Now, our main task is to automatically learn the similarities/differences between our reference and our landmark, right? We take the first pair: `discussion thrive` vs. `discussion digress`. We have a vector representation for each of these phrases.

**How do we compare two vectors?**

We use cosine similarity to calculate a single score that would tell us about the similarity between these two vectors. The bigger the cosine, the more similar two vectors are. Cosine ranges from 0 to 1 (0 is very low similarity, 1 is very high similarity). Let's say, we get a cosine of 0.89, it means that according to the multiplicative model (remember, we decided to use multiplicative semantic composition method), these phrases are very similar (cosine is quite high). But wait a second, we know that these two phrases should be of low similarity, right? Because this is what the value in `hilo` tells us about this pair - `discussion thrive` and `discussion digress` are not similar to each other. Clearly, our multiplicative method fails to capture it.

**What can we do to improve our model?**

We can try a different composition method to get a phrase vector from phrase's words: let's replace multiplication with addition. Or we can also use combined method. Let's say we used combined method and run cosine again; this time it tells that the cosine score is 0.45. Ok, this seems to be quite low, and it also agrees with our knowledge that these phrases are indeed not similar.

---

In other words, we need to evaluate different composition models (additive, multiplicative, combined) and analyse how well they perform. **How do we analyse their performance?** Because we have the ground-truth for comparison (`hilo` values), we know whether our phrases are actually similar or not. We want our cosine score to reflect this knowledge: if the score is high, but the groun-truth hilo is low, then we have a problem in the model - it did not learn things well, we need to replace the composition function.

---

_Long story short_: `hilo` is something that we compare our cosine to. `hilo` contains correct answers about similarity between phrases, and cosine should agree with this. If reference-landmark pair are `high` in `hilo`, then cosine should be high enough to reflect that. If cosine is not high in this case, then we look at our model and change the composition function. We need to find the function, which give us cosines that are super close to the ground-truth known `hilo` values.

<img src="res.png" alt="drawing" width="500"/>

Next, we want to compute do we compute correlation between our model's predictions and the ground-truth, something similar to the results from [2] (image above):

---

In `High` and `Low` columns we have mean cosine values.
These are calculated by averaging cosine scores for all pairs of phrases per model.
Rows introduce different models: `add` is additive, `multiply` is multiplicative, etc. (these are all described in the paper). `NonComp` is a baseline model, the most "stupid" one, it should be the worst. `UpperBound` is how humans performed in this task (they were asked to rate similarity between pairs of phrases, 1 is the lowest, 7 is the highest). Why these numbers are in a different scale, not from 0 to 1 like cosine, but from 1 to 7? Because they are not normalised, and authors explicitly said that they are interested in relative differences.

We need models which are closer to human ratings. `Add` model has 0.59 mean for `High` and 0.59 for `Low`, so it did not learn to differentiate between high similarity pairs and low similarity ones. This is a bad model then, we need a better one. `WeightAdd` seems to be doing better, the difference between `High` and `Low` is now 0.01, but it's also quite bad - the difference is not that obvious. The best models are `Multiply` and `Combined`, because their mean cosines for `High` and `Low` are quite different from each other. We can see that these two models gave higher cosines for high similarity pairs (0.42 and 0.38), while giving lower cosines for low similarity pairs (0.28 and 0.28). And this is a good result - it shows that these two composition functions are so far the best in (i) giving high cosine go highly similar pairs and low cosine to very dissimilar pairs, and (ii) keeping the distance (range) between high and low cosines quite large.

**However, we can't say how far/close they are when compared to human performance (UpperBound) since human scores are not normalised.**

Still, we want to choose a model which is the closest to humans. This is why we want to run **the correlation test.**

How do we perform the correlation test? We need to see how well *each model* correlates with human judgements. So for each model, we would have a vector of cosine values this model gives for each pair that we have. For example, let's say we have three pairs of phrases and our cosine values from additive model are the following ones: `[0.89, 0.40, 0.70]`. Now we need to get a vector of the same size, but for human scores. What do we have for human scores? We have multiple participants, which means that a single phrase can be evaluated by multiple participants. Let's say, the first phrase has scores from two participants (`6, 7`), so what we would do is that we would average it to have a single number (`(6 + 7) / 2 = 6.5`). With this, we can get a mean vector of human scores per item: `[6.5, 3, 6]`.

Now, we have two vectors and we can run Spearman correlation on these vectors. This is what exactly what the third column in the Results table is showing (the correlation value). There is also a p-value (denoted right below the table).

**What is your ultimate task in this part of the assignment?**

(i) Process the dataset and extract `reference - landmark` pairs; you can use the code from the lecture as something to start with. Try to keep information about human rating (`input`) and high/low similarity (`hilo`), because you will need it for correlation tests. Also, you might want to keep the information about participant id (will be useful for getting average numbers for correlation tests). Which format you should use to keep all this data? It's up to you, but a dictionary-like format could be a good choice.

(ii) Build models of semantic phrase composition: in the lecture we introduced simple additive, simple multiplicative and combined models (details are in [2]). Your task is to take a single pair of phrases, and compute the composition of its vectors using each of these functions. Thus, you will have (at least) three compositional models that take each `noun - verb` phrase from the pair (these phrases can be either references or landmarks) and output a single vector, representing the meaning of this phrase. As your semantic space, you can use pretrained spaces (standard space, ppmi or svd) introduced above. It is up to you which space you use, but for someone who runs your code, it should be pretty straightforward to switch between them.

(iii) calculate Spearman correlation between each model's predictions and human judgements; you should have something similar to the scores that are shown in the paper.

**Thoughts process behind calculating the correlation:**

Let's look at the example pair: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. Let's say we have 3 humans evaluating the similarity between these two phrases and we combine their scores into a single vector: `[5, 6, 5]`. We need to average them to get our human vector for correlation: `[5.3]`.

Our A model's output:  
`cosine(p1, p2) = 0.88`, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  

Therefore, we have human rating vector `[5.3]` and model A output `[0.88]`. Next is to compute correlation between these two vectors. This should give you a correlation value and p-value for the model of choice and human ratings.

Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]). Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.

---

**The minimum to do in this task**: compute correlations for at least _ONE_ model and human ratings. However, this should not be hard to run it for any other model as well. For examples on how to interpret the results, look at Section 5 Results of the original paper.

In [17]:
our_dataset = space_10k

In [18]:
# load the task dataset
with open('./mitchell_lapata_acl08.txt', 'r') as f:
    phrase_dataset = f.read().splitlines()

for line in phrase_dataset[:10]:
    print(line)

# get all unique words
words = []
for line in phrase_dataset[1:]:
    _, verb, noun, landmark, _, _ = line.split()
    if verb not in words:
        words.append(verb)
    if noun not in words:
        words.append(noun)
    if landmark not in words:
        words.append(landmark)

participant verb noun landmark input hilo
participant20 stray thought roam 7 low
participant20 stray discussion digress 6 high
participant20 stray eye roam 7 high
participant20 stray child digress 1 low
participant20 throb body pulse 5 high
participant20 throb head shudder 2 low
participant20 throb voice shudder 3 low
participant20 throb vein pulse 6 high
participant20 chatter machine click 4 high


In [19]:
# simply check if all words that we have in our task dataset can be found in the reference corpus (the result should return nothing)
to_remove = []
for w in words:
    if w not in our_dataset:
        print(w)
        to_remove.append(w)
# if something is not found, makes sense to ignore phrases with such non-present words

stray
roam
digress
throb
pulse
shudder
vein
chatter
gabble
tooth
rebound
ricochet
optimism
flicker
waver
flick
subside
lessen
symptom
slump
slouch
stoop
erupt
burst
temper
flare
recoil
flinch
prosper
fluctuate
falter
cigarette
reel
whirl
stagger
glow
cigar


In [20]:
# cleaning the task dataset (we might call it phrase dataset from now on)
# we are removing all phrases which contain non-found words
# this would probably remove other words as well (those, which are paired with the non-found words)

cleaned_phrase_dataset = []
for line in phrase_dataset:
    _, verb, noun, landmark, _, _ = line.split()
    if verb in to_remove or noun in to_remove or landmark in to_remove:
        continue
    cleaned_phrase_dataset.append(line)

target_words = []
for line in cleaned_phrase_dataset[1:]:
    _, verb, noun, landmark, _, _ = line.split()
    if verb not in target_words:
        target_words.append(verb)
    if noun not in target_words:
        target_words.append(noun)
    if landmark not in target_words:
        target_words.append(landmark)

In [21]:
#our code:
print(cleaned_phrase_dataset)

['participant verb noun landmark input hilo', 'participant20 bow butler submit 3 low', 'participant20 bow company submit 5 high', 'participant20 boom sale thunder 3 low', 'participant20 boom gun thunder 6 high', 'participant21 bow butler submit 2 low', 'participant21 bow company submit 6 high', 'participant21 boom gun thunder 6 high', 'participant21 boom sale thunder 5 low', 'participant22 bow butler submit 2 low', 'participant22 bow company submit 6 high', 'participant22 boom gun thunder 7 high', 'participant22 boom sale thunder 2 low', 'participant23 bow butler submit 3 low', 'participant23 bow company submit 4 high', 'participant23 boom sale thunder 3 low', 'participant23 boom gun thunder 5 high', 'participant24 bow butler submit 4 low', 'participant24 bow company submit 6 high', 'participant24 boom gun thunder 6 high', 'participant24 boom sale thunder 1 low', 'participant25 bow butler submit 5 low', 'participant25 bow company submit 5 high', 'participant25 boom gun thunder 6 high',

In [22]:
def add(verb, noun):
    weight_verb = 0.8
    weight_noun = 1 - weight_verb

    return weight_verb * verb + weight_noun * noun


def multiply(verb, noun):
    return verb * noun


def combined(verb, noun):
    weight_verb = 0.95
    weight_noun = 0
    weight_mul = 0.05

    return weight_verb * verb + weight_noun * noun + weight_mul * verb * noun


composition_methods = [add, multiply, combined]

In [23]:
from scipy.spatial.distance import cosine

"""
grouped = {
    'low': {
        (verb, noun, landmark): {
            add: add_cosine,
            multiply: multiply_cosine,
            combined: combined_cosine,
            human: [participant1_input, participant2_input, ...]
        }
    }
}
"""

grouped = {
    'high': {},
    'low': {}
}

for phrase in cleaned_phrase_dataset[1:]:
    _, verb, noun, landmark, input, hilo = phrase.split()

    combination = (verb, noun, landmark)

    if combination not in grouped[hilo]:
        grouped[hilo][combination] = {'human': []}
        for composition_method in composition_methods:
            v1 = composition_method(our_dataset[verb], our_dataset[noun])
            v2 = composition_method(our_dataset[landmark], our_dataset[noun])
            grouped[hilo][combination][composition_method] = 1 - cosine(v1, v2)

    grouped[hilo][combination]['human'].append(int(input))

In [24]:
import pandas as pd

results = [['human']]

all_human = []
for hifi in grouped:
    human = [sum(grouped_values['human']) / len(grouped_values['human']) for grouped_values in grouped[hifi].values()]
    all_human.extend(human)
    results[-1].append(sum(human) / len(human))
rho, pval = stats.spearmanr(all_human, all_human)
results[-1].extend([rho, pval])

for composition_method in composition_methods:
    results.append([composition_method.__name__])
    complete_hifi = []
    for hifi in grouped:
        model = [grouped_values[composition_method] for grouped_values in grouped[hifi].values()]
        complete_hifi.extend(model)
        results[-1].append(sum(model) / len(model))
    rho, pval = stats.spearmanr(complete_hifi, all_human)
    results[-1].extend([rho, pval])


df = pd.DataFrame(results, columns=['model', 'high', 'low', 'rho', 'pval'])
df = np.round(df, decimals=3)
df

,model,high,low,rho,pval
0,human,5.360,2.943,1.000,0.000
1,add,0.942,0.861,0.405,0.320
2,multiply,0.908,0.896,0.167,0.693
3,combined,0.903,0.879,0.286,0.493


**Any comments/thoughts should go here:**

The scores are quite different from the paper. This is very likely due to the fact that the dataset is very small. We only have 8 different combinations (4 low, 4 high), since the remaining words didn't appear in the pretrained models. Having such a small dataset, the scores are not really meaningful and comparable to the paper.

Assuming, they were meaningful, it can be seen that the (weighted) addition is the closest to human judgement. For the *svd* and *ppmi*, it is followed by the multiplication method. The combination produces the least similar results to human judgement. For the *raw* space, the combined method performs slightly better than the multiplication method.

In further experiments, the weights would need to be adjusted. In our cases, we just chose the best performing weights from the paper.

# Literature

  - [1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

  - [2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
  - [3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

## Marks

This assignment has a total of 60 marks.